In [1]:
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler
import torch.nn as nn

import torchvision
from torchvision import transforms, datasets, models
import torch.nn.functional as F

import numpy as np
import ast

import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from PIL import Image
from timeit import default_timer as timer

In [270]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [236]:
# Image transformations (from tutorial)
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'val':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Test does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [237]:
class MovieAttributeDataset(Dataset):
    """Custom PyTorch Dataset for movie data 
    """
    def __init__(self, annotations_file, img_dir, transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path)
        budget = self.img_labels.iloc[idx, 1]
        genres = torch.Tensor(ast.literal_eval(self.img_labels.iloc[idx, 2]))
        if self.transform:
            image = self.transform(image)
        return image, budget, genres

In [238]:
train_dataset = MovieAttributeDataset(annotations_file="train_set_15_shots.csv", img_dir="", transform=image_transforms["train"])
# train_dataset = MovieAttributeDataset(annotations_file="train_set_all_shots.csv", img_dir="", transform=image_transforms["train"])
val_dataset = MovieAttributeDataset(annotations_file="val_set.csv", img_dir="", transform=image_transforms["val"])

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [71]:
# Visualize data in each training batch
for images, budgets, genres in train_dataloader:
    print(images.shape) # (batch_size, color_channels, height, width)
    print(budgets.shape)
    print(genres.shape)
    break

torch.Size([64, 3, 224, 224])
torch.Size([64])
torch.Size([64, 22])


In [46]:
# Modified from tutorial
def train(model,
          criterion1,
          criterion2,
          optimizer,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=20,
          print_every=2):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_budget_acc = 0
        train_genre_acc = 0
        valid_budget_acc = 0
        valid_genre_acc = 0

        # Set to training
        model.train()
        start = timer()

        ## training step
        for ii, (images, budgets, genres) in enumerate(train_loader):

            images = images.to(device)
            budgets = budgets.to(device)
            genres = genres.to(device)

            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are non-log probabilities
            budget_pred, genre_pred = model(images)
            # 64 x 22

            # Loss and backpropagation of gradients
            loss1 = criterion1(budget_pred, budgets)
            loss2 = criterion2(genre_pred, genres)
            loss = loss1 + loss2
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * images.size(0)

            # Calculate accuracy by finding max log probability
            _, pred1 = torch.max(budget_pred, dim=1)
            correct_tensor = pred1.eq(budgets.data.view_as(pred1))

            _, idx = genre_pred.topk(3, dim=1)
            pred2 = torch.zeros_like(genre_pred)
            pred2[torch.arange(genres.size(0)).unsqueeze(1), idx] = 1
            correct_tensor2 = pred2.eq(genres.data.view_as(pred2))
            
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            accuracy2 = torch.mean(correct_tensor2.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_budget_acc += accuracy.item() * images.size(0)
            train_genre_acc += accuracy2.item() * genres.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for images, budgets, genres in valid_loader:
                    # Tensors to gpu
                    images = images.to(device)
                    budgets = budgets.to(device)
                    genres = genres.to(device)

                    # Forward pass
                    budget_pred, genre_pred = model(images)

                    # Validation loss
                    loss1 = criterion1(budget_pred, budgets)
                    loss2 = criterion2(genre_pred, genres)
                    loss = loss1 + loss2
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * images.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(budget_pred, dim=1)
                    correct_tensor = pred.eq(budgets.data.view_as(pred))

                    _, idx = genre_pred.topk(3, dim=1)
                    pred2 = torch.zeros_like(genre_pred)
                    pred2[torch.arange(genres.size(0)).unsqueeze(1), idx] = 1
                    correct_tensor2 = pred2.eq(genres.data.view_as(pred2))

                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    accuracy2 = torch.mean(correct_tensor2.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_budget_acc += accuracy.item() * images.size(0)
                    valid_genre_acc += accuracy2.item() * genres.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_budget_acc = train_budget_acc / len(train_loader.dataset)
                train_genre_acc = train_genre_acc / len(train_loader.dataset)
                valid_budget_acc = valid_budget_acc / len(valid_loader.dataset)
                valid_genre_acc = valid_genre_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_budget_acc, valid_budget_acc, train_genre_acc, valid_genre_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tBudget Training Accuracy: {100 * train_budget_acc:.2f}%\t Budget Validation Accuracy: {100 * valid_budget_acc:.2f}%'
                    )
                    print(
                        f'\t\tGenre Training Accuracy: {100 * train_genre_acc:.2f}%\t Genre Validation Accuracy: {100 * valid_genre_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_budget_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_best_acc:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_budget_acc', 
                                'valid_budget_acc', 'train_genre_acc', 'valid_genre_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_best_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_budget_acc', 'valid_budget_acc', 'train_genre_acc', 'valid_genre_acc'])
    return model, history

## Custom model training

In [221]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # m x m x 3 image
        # 224x224x3 => 222x222x32, kernel: k -> m - k + 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

        self.d = nn.Sequential(
            nn.Linear(7 * 7 * 32, 256), nn.ReLU(), nn.Dropout(0.4),
            nn.Linear(256, 256), nn.ReLU(), nn.Dropout(0.4),
            nn.Linear(256, 256), nn.ReLU(), nn.Dropout(0.4)
        )

        self.o1 = nn.Linear(256, 5)
        self.o2 = nn.Linear(256, 22)

    def forward(self, x):
        # 64x3x224x224 => 32x32x222x222
        x = self.conv1(x)
        x = F.relu(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.maxpool2(x)
        x = self.avgpool(x)

        # flatten => 64 x (32*222*222)
        x = x.flatten(start_dim = 1)

        # 64 x (32*222*222) => 64x256
        x = self.d(x)

        # logits (the raw, unnormalized predictions) => 64x5
        out = self.o1(x)
        # out = F.softmax(logits, dim=1) # turn logits into a set of probabilities.

        out2 = self.o2(x)
        # out2 = F.sigmoid(logits2, dim=1)
        return out, out2

In [222]:
learning_rate = 0.05
criterion = nn.CrossEntropyLoss()
criterion2 = nn.BCEWithLogitsLoss(reduction='mean') # binary cross entropy loss + softmax

In [223]:
model = MyModel()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train(model, criterion, criterion2, optimizer, train_dataloader, val_dataloader, "test.pt", 3, 10, 2)

Starting Training from Scratch.

Epoch: 1	100.00% complete. 19.53 seconds elapsed in epoch.
Epoch: 1 	Training Loss: 1.7255 	Validation Loss: 1.1814
		Budget Training Accuracy: 41.75%	 Budget Validation Accuracy: 55.56%
		Genre Training Accuracy: 84.43%	 Genre Validation Accuracy: 84.85%
Epoch: 3	100.00% complete. 21.59 seconds elapsed in epoch.
Epoch: 3 	Training Loss: 1.7200 	Validation Loss: 1.1976
		Budget Training Accuracy: 41.80%	 Budget Validation Accuracy: 55.56%
		Genre Training Accuracy: 84.43%	 Genre Validation Accuracy: 84.85%

Early Stopping! Total epochs: 3. Best epoch: 0 with loss: 1.18 and acc: 55.56%
84.89 total seconds elapsed. 21.22 seconds per epoch.


(MyModel(
   (conv1): Conv2d(3, 32, kernel_size=(2, 2), stride=(1, 1))
   (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
   (conv2): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1))
   (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
   (avgpool): AdaptiveAvgPool2d(output_size=(7, 7))
   (d): Sequential(
     (0): Linear(in_features=1568, out_features=256, bias=True)
     (1): ReLU()
     (2): Dropout(p=0.4, inplace=False)
     (3): Linear(in_features=256, out_features=256, bias=True)
     (4): ReLU()
     (5): Dropout(p=0.4, inplace=False)
     (6): Linear(in_features=256, out_features=256, bias=True)
     (7): ReLU()
     (8): Dropout(p=0.4, inplace=False)
   )
   (o1): Linear(in_features=256, out_features=5, bias=True)
   (o2): Linear(in_features=256, out_features=22, bias=True)
 ),
    train_loss  valid_loss  train_budget_acc  valid_budget_acc  \
 0   33.254691    1.176496          0.300723          0.55

## Resnet18 pretrained model training

In [262]:
class ResnetPretrainedModel(nn.Module):
    def __init__(self, resnet):
        super(ResnetPretrainedModel, self).__init__()
        self.resnet = resnet
        self.resnet.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(256, 256),
            nn.ReLU(True),
            nn.Dropout()
        )

        self.output1 = nn.Linear(256, 5)
        self.output2 = nn.Linear(256, 22)

    def forward(self, x):
        x = self.resnet(x)
        output1 = self.output1(x)
        output2 = self.output2(x)
        return output1, output2

In [263]:
resnet18 = models.resnet18(weights='ResNet18_Weights.DEFAULT')

for param in resnet18.parameters():
    param.requires_grad = False

model3 = ResnetPretrainedModel(resnet18)
model3 = model3.to(device)

# for name,param in model3.named_parameters():
#     if param.requires_grad == True:
#         print("\t",name)

optimizer = torch.optim.Adam(model3.parameters(), lr=learning_rate)
train(model3, criterion, criterion2, optimizer, train_dataloader, val_dataloader, "test3.pt", 3, 10, 2)

Starting Training from Scratch.

Epoch: 1	100.00% complete. 20.44 seconds elapsed in epoch.
Epoch: 1 	Training Loss: 1.7486 	Validation Loss: 1.2047
		Budget Training Accuracy: 40.86%	 Budget Validation Accuracy: 55.56%
		Genre Training Accuracy: 84.43%	 Genre Validation Accuracy: 84.85%
Epoch: 3	100.00% complete. 19.73 seconds elapsed in epoch.
Epoch: 3 	Training Loss: 1.7094 	Validation Loss: 1.1784
		Budget Training Accuracy: 41.80%	 Budget Validation Accuracy: 55.56%
		Genre Training Accuracy: 84.43%	 Genre Validation Accuracy: 84.85%

Early Stopping! Total epochs: 3. Best epoch: 0 with loss: 1.16 and acc: 55.56%
86.67 total seconds elapsed. 21.67 seconds per epoch.


(ResnetPretrainedModel(
   (resnet): ResNet(
     (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
     (layer1): Sequential(
       (0): BasicBlock(
         (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (relu): ReLU(inplace=True)
         (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
       (1): BasicBlock(
         (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, moment

## Vgg16 pretrained model training

In [249]:
class VggPretrainedModel(nn.Module):
    def __init__(self, vgg_features):
        super(VggPretrainedModel, self).__init__()
        self.features = vgg_features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.fc = nn.Sequential(
            nn.Linear(512 * 7 * 7, 256),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(256, 256),
            nn.ReLU(True),
            nn.Dropout()
        )
        
        self.output1 = nn.Linear(256, 5)
        self.output2 = nn.Linear(256, 22)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        output1 = self.output1(x)
        output2 = self.output2(x)
        return output1, output2

In [257]:
vgg16 = models.vgg16(weights='VGG16_Weights.DEFAULT')

# Step 2: Freeze parameters of pre-trained layers
for param in vgg16.parameters():
    param.requires_grad = False

# Extract features from VGG16
features = vgg16.features

model2 = VggPretrainedModel(features)
model2 = model2.to(device)

# for name,param in model2.named_parameters():
#     if param.requires_grad == True:
#         print("\t",name)

optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)
train(model2, criterion, criterion2, optimizer, train_dataloader, val_dataloader, "test2.pt", 3, 10, 2)

	 fc.0.weight
	 fc.0.bias
	 fc.3.weight
	 fc.3.bias
	 output1.weight
	 output1.bias
	 output2.weight
	 output2.bias
Starting Training from Scratch.

Epoch: 1	100.00% complete. 52.62 seconds elapsed in epoch.
Epoch: 1 	Training Loss: 39.8595 	Validation Loss: 1.1909
		Budget Training Accuracy: 40.91%	 Budget Validation Accuracy: 55.56%
		Genre Training Accuracy: 84.14%	 Genre Validation Accuracy: 84.78%
Epoch: 3	100.00% complete. 52.16 seconds elapsed in epoch.
Epoch: 3 	Training Loss: 31.6327 	Validation Loss: 1.1751
		Budget Training Accuracy: 41.80%	 Budget Validation Accuracy: 55.56%
		Genre Training Accuracy: 84.21%	 Genre Validation Accuracy: 84.85%

Early Stopping! Total epochs: 3. Best epoch: 0 with loss: 1.17 and acc: 55.56%
224.39 total seconds elapsed. 56.10 seconds per epoch.


(VggPretrainedModel(
   (features): Sequential(
     (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (1): ReLU(inplace=True)
     (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (3): ReLU(inplace=True)
     (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (6): ReLU(inplace=True)
     (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (8): ReLU(inplace=True)
     (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (11): ReLU(inplace=True)
     (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (13): ReLU(inplace=True)
     (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (15): ReLU(inplace=True)
     (16): MaxPool2d(kernel_size=

## Testing code

In [241]:
def test(model, test_dataloader):
    budget_acc = 0
    genre_acc = 0
    for (images, budgets, genres) in test_dataloader:
        out1, out2 = model(images)

        budget_pred = torch.argmax(out1, dim=1)
        correct = budget_pred.eq(budgets.data.view_as(budget_pred))

        _, idx = out2.topk(3, dim=1)
        genre_pred = torch.zeros_like(out2)
        genre_pred[torch.arange(genres.size(0)).unsqueeze(1), idx] = 1
        correct2 = genre_pred.eq(genres.data.view_as(genre_pred))
        # Need to convert correct tensor from int to float to average
        budget_accuracy = torch.mean(correct.type(torch.FloatTensor))
        genre_accuracy = torch.mean(correct2.type(torch.FloatTensor))
        # Multiply average accuracy times the number of examples in batch
        budget_acc += budget_accuracy.item() * images.size(0)
        genre_acc += genre_accuracy.item() * genres.size(0)
    
    budget_acc = budget_acc / len(test_dataloader.dataset)
    genre_acc = genre_acc / len(test_dataloader.dataset)
    print(
        f'Budget Accuracy: {100 * budget_acc:.2f}%\t Genre Accuracy: {100 * genre_acc:.2f}%'
    )

In [242]:
test_dataset = MovieAttributeDataset(annotations_file="test_set_15_shots.csv", img_dir="", transform=image_transforms["test"])
# test_dataset = MovieAttributeDataset(annotations_file="test_set_all_shots.csv", img_dir="", transform=image_transforms["test"])
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [267]:
final = MyModel()
final.load_state_dict(torch.load('test.pt'))
test(final, test_dataloader)

Budget Accuracy: 33.33%	 Genre Accuracy: 81.82%


In [266]:
final3 = ResnetPretrainedModel(resnet18)
final3 = final3.to('cpu')
final3.load_state_dict(torch.load('test3.pt'))
test(final3, test_dataloader)

Budget Accuracy: 33.33%	 Genre Accuracy: 81.82%


In [268]:
final2 = VggPretrainedModel(features)
final2 = final2.to('cpu')
final2.load_state_dict(torch.load('test2.pt'))
test(final2, test_dataloader)

Budget Accuracy: 28.89%	 Genre Accuracy: 81.75%
